In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
%%time
toxic_comments_df = pd.read_csv('train.csv',nrows=50000)
toxic_comments_df =  toxic_comments_df[['id','target','comment_text']]


In [ ]:
#creating our truth column. comment is toxic if target >= .5
toxic_comments_df['truth'] = toxic_comments_df.target.apply(lambda x: 1 if x>=.5 else 0 ).astype('float32')
toxic_comments_df.head()


In [ ]:
toxic_comments_df['InputData'] = toxic_comments_df['comment_text']
#toxic_comments_df['InputData'] = toxic_comments_df['comment_text'].str.replace('[{}]'.format(string.punctuation), '')
toxic_comments_df['InputData'] = toxic_comments_df['InputData'].apply(lambda x: " ".join([ word.lower() if word[0].isalpha() else ""  for word in str(x).split()]))

toxic_comments_df.head()

In [ ]:
X = toxic_comments_df.InputData
Y = toxic_comments_df.truth
le = LabelEncoder()
Y = le.fit_transform(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)


In [ ]:
%%time
max_words = 2000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=256,epochs=10,
          validation_split=0.4,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])